In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import PIL
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import FloatVGG16
from SampleNN import *
from tqdm import tqdm_notebook
%matplotlib inline
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
import torchvision
import SampleVGG16

In [2]:
import torchvision.transforms as transforms
traindir = '/home/data/yanzy/train'
valdir = '/home/data/yanzy/val'
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

val_dataset =  torchvision.datasets.ImageFolder(
    valdir,
    transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize,
]))
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=64,
                                         shuffle=False, num_workers = 32, pin_memory=False)

train_dataset =  torchvision.datasets.ImageFolder(
    traindir,
    transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize,
]))
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=8,
                                         shuffle=True, num_workers = 0, pin_memory=False)

In [3]:
def val(Sample = 1, N = 16, m = 6):
    TMP = protectStateDict(sample_net)
    sampleStateDict(sample_net, N, m)
    sample_net.eval()
    correct = 0
    total = 0
    count = 0
    with torch.no_grad():
        for data in tqdm_notebook(val_loader, desc = "m = %d"%(m), leave = False):
            if count % Sample == 0:
            
                images, labels = data
                images, labels = images.to(device), labels.to(device)
                images = oSample()(images)
                outputs = sample_net(images)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)


                correct += (predicted == labels).sum().item()
                #print correct, total
            count += 1

    print('When m = %d, Accuracy of the network on the %d test images: %.3f %%' % (m, total,
        100.0 * correct / total))
    
    sample_net.load_state_dict(TMP)
    return 100.0 * correct / total

In [12]:
float_net = torchvision.models.vgg16_bn(pretrained=True).to(device)
gt  = float_net.state_dict()
b = gt.keys()
N = 8
for m in range(4,6):
    sample_net = SampleVGG16.SampleVGG16(N = N, m = m, bn = True).to(device)
    lol = sample_net.state_dict()
    a = lol.keys()
    for i in range(len(a)):
        lol[a[i]] = gt[b[i]]
    sample_net.load_state_dict(lol)
    val(Sample = 1, N = N, m = m)

When m = 4, Accuracy of the network on the 50000 test images: 0.122 %


When m = 5, Accuracy of the network on the 50000 test images: 0.490 %


In [4]:
N = 16
m = 5
float_net = torchvision.models.vgg16_bn(pretrained=True).to(device)
gt  = float_net.state_dict()
b = gt.keys()
sample_net = SampleVGG16.SampleVGG16(N = N, m = m, bn = True).to(device)
lol = sample_net.state_dict()
a = lol.keys()
for i in range(len(a)):
    lol[a[i]] = gt[b[i]]

sample_net.load_state_dict(lol)

In [5]:
criterion_s = nn.CrossEntropyLoss()
#optimizer_s = optim.SGD(sample_net.parameters(), lr=1e-6, momentum=0.9)
optimizer_s = optim.Adam(sample_net.parameters(), lr=1e-5, weight_decay=1e-6)
def train(epoch, Sample = 10000):
    sample_net.train()
    running_loss = 0.0
    count = 0
    for i, data in tqdm_notebook(enumerate(train_loader, 0),desc = "epoch = %d"%(epoch)):
        # get the inputs
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        # zero the parameter gradients
        optimizer_s.zero_grad()

        # forward + backward + optimize
        TMP = protectStateDict(sample_net)
        sampleStateDict(sample_net)
        outputs = sample_net(inputs)
        loss = criterion_s(outputs, labels)
        loss.backward()
        sample_net.load_state_dict(TMP)
        del TMP
        torch.cuda.empty_cache()
        optimizer_s.step()
        count += labels.size(0)
        if count >= Sample:
            break

        # print statistics
        running_loss += loss.item()
    print('[%d, %5d] loss: %.3f' %
          (epoch + 1, i + 1, running_loss / 1000))
    return running_loss

In [6]:
for epoch in range(4):

    torch.cuda.empty_cache()
    train(epoch, Sample = 10000000)
    torch.cuda.empty_cache()
    val(Sample = 1, N = N, m = m)

RuntimeError: CUDA error: out of memory

1615